In [1]:
import os,math
from keras.layers import Dropout, Dense, SimpleRNN
from sklearn.preprocessing import MinMaxScaler
import sklearn
from sklearn import metrics
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
# 支持中文
plt.rcParams['font.sans-serif'] = ['SimHei']  # 用来正常显示中文标签
plt.rcParams['axes.unicode_minus'] = False  # 用来正常显示负号
data = pd.read_csv("SH600519.csv")  # 读取股票文件
training_set = data.iloc[0:2426 - 300, 2:3].values  


test_set = data.iloc[2426 - 300:, 2:3].values  
sc           = MinMaxScaler(feature_range=(0, 1))
training_set = sc.fit_transform(training_set)
test_set     = sc.transform(test_set) 
x_train = []
y_train = []

x_test = []
y_test = []
for i in range(60, len(training_set)):
    x_train.append(training_set[i - 60:i, 0])
    y_train.append(training_set[i, 0])
    
for i in range(60, len(test_set)):
    x_test.append(test_set[i - 60:i, 0])
    y_test.append(test_set[i, 0])
    
# 对训练集进行打乱
np.random.seed(7)
np.random.shuffle(x_train)
np.random.seed(7)
np.random.shuffle(y_train)
tf.random.set_seed(7)
x_train, y_train = np.array(x_train), np.array(y_train) # x_train形状为：(2066, 60, 1)
x_test,  y_test  = np.array(x_test),  np.array(y_test)

"""
输入要求：[送入样本数， 循环核时间展开步数， 每个时间步输入特征个数]
"""
x_train = np.reshape(x_train, (x_train.shape[0], 60, 1))
x_test  = np.reshape(x_test,  (x_test.shape[0], 60, 1))

model = tf.keras.Sequential([
    SimpleRNN(100, return_sequences=True), #布尔值。是返回输出序列中的最后一个输出，还是全部序列。
    Dropout(0.1),                         #防止过拟合
    SimpleRNN(100),
    Dropout(0.1),
    Dense(1)
])

model.compile(optimizer=tf.keras.optimizers.Adam(0.001),
              loss='mean_squared_error')  # 损失函数用均方误差
history = model.fit(x_train, y_train, 
                    batch_size=64, 
                    epochs=20, 
                    validation_data=(x_test, y_test), 
                    validation_freq=1)                  #测试的epoch间隔数

model.summary()


Epoch 1/20
33/33 [==============================] - 3s 39ms/step - loss: 0.1262 - val_loss: 0.0670
Epoch 2/20
33/33 [==============================] - 1s 30ms/step - loss: 0.0172 - val_loss: 0.0786
Epoch 3/20
33/33 [==============================] - 1s 29ms/step - loss: 0.0126 - val_loss: 0.0037
Epoch 4/20
33/33 [==============================] - 1s 29ms/step - loss: 0.0089 - val_loss: 0.0160
Epoch 5/20
33/33 [==============================] - 1s 29ms/step - loss: 0.0064 - val_loss: 0.0352
Epoch 6/20
33/33 [==============================] - 1s 29ms/step - loss: 0.0052 - val_loss: 0.0207
Epoch 7/20
33/33 [==============================] - 1s 29ms/step - loss: 0.0048 - val_loss: 0.0307
Epoch 8/20
33/33 [==============================] - 1s 29ms/step - loss: 0.0046 - val_loss: 0.0255
Epoch 9/20
33/33 [==============================] - 1s 29ms/step - loss: 0.0037 - val_loss: 0.0246
Epoch 10/20
33/33 [==============================] - 1s 29ms/step - loss: 0.0031 - val_loss: 0.0069
Epoch 11/

In [ ]:

plt.plot(history.history['loss']    , label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Training and Validation Loss by K同学啊')
plt.legend()
plt.show()

predicted_stock_price = model.predict(x_test)                       # 测试集输入模型进行预测
predicted_stock_price = sc.inverse_transform(predicted_stock_price) # 对预测数据还原---从（0，1）反归一化到原始范围
real_stock_price = sc.inverse_transform(test_set[60:])              # 对真实数据还原---从（0，1）反归一化到原始范围

# 画出真实数据和预测数据的对比曲线
plt.plot(real_stock_price, color='red', label='Stock Price')
plt.plot(predicted_stock_price, color='blue', label='Predicted Stock Price')
plt.title('Stock Price Prediction by K同学啊')
plt.xlabel('Time')
plt.ylabel('Stock Price')
plt.legend()
plt.show()

In [2]:
from sklearn.model_selection import GridSearchCV
from keras.wrappers.scikit_learn import KerasRegressor

# 定义创建模型的函数
def create_model(units=100, dropout_rate=0.1, learning_rate=0.001):
    model = tf.keras.Sequential([
        SimpleRNN(units, return_sequences=True),
        Dropout(dropout_rate),
        SimpleRNN(units),
        Dropout(dropout_rate),
        Dense(1)
    ])
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate),
                  loss='mean_squared_error')
    return model

# 创建 KerasRegressor
model = KerasRegressor(build_fn=create_model, epochs=20, batch_size=64, verbose=0)

# 定义搜索的参数网格
param_grid = {
    'units': [50, 100, 150],
    'dropout_rate': [0.1, 0.2, 0.3],
    'learning_rate': [0.001, 0.01, 0.1]
}

# 执行网格搜索
grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=3, scoring='neg_mean_squared_error')
grid_result = grid_search.fit(x_train, y_train)

# 输出结果
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

# 最佳模型
best_model = grid_result.best_estimator_.model


C:\Users\gloomy\AppData\Local\Temp\ipykernel_40384\2521160233.py:18: DeprecationWarning: KerasRegressor is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  model = KerasRegressor(build_fn=create_model, epochs=20, batch_size=64, verbose=0)


Best: -0.000213 using {'dropout_rate': 0.3, 'learning_rate': 0.001, 'units': 100}
-0.000247 (0.000032) with: {'dropout_rate': 0.1, 'learning_rate': 0.001, 'units': 50}
-0.000274 (0.000118) with: {'dropout_rate': 0.1, 'learning_rate': 0.001, 'units': 100}
-0.000327 (0.000184) with: {'dropout_rate': 0.1, 'learning_rate': 0.001, 'units': 150}
-0.002102 (0.001247) with: {'dropout_rate': 0.1, 'learning_rate': 0.01, 'units': 50}
-0.027256 (0.034163) with: {'dropout_rate': 0.1, 'learning_rate': 0.01, 'units': 100}
-0.086417 (0.022608) with: {'dropout_rate': 0.1, 'learning_rate': 0.01, 'units': 150}
-0.057893 (0.024643) with: {'dropout_rate': 0.1, 'learning_rate': 0.1, 'units': 50}
-0.096409 (0.023201) with: {'dropout_rate': 0.1, 'learning_rate': 0.1, 'units': 100}
-0.100457 (0.007460) with: {'dropout_rate': 0.1, 'learning_rate': 0.1, 'units': 150}
-0.000416 (0.000133) with: {'dropout_rate': 0.2, 'learning_rate': 0.001, 'units': 50}
-0.000318 (0.000049) with: {'dropout_rate': 0.2, 'learning_ra